<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## DataFrame Manipulation Lab with Chipotle Data

_Authors: Kiefer Katovich (SF)_

---

This lab is intended to cover a variety of skills for data manipulation in pandas with a challenging dataset.

In addition to python function-writing practice, you will be practicing multiple pandas EDA skills including:
- Data cleaning
- Grouping
- Data summarization and aggregation
- [Pandas split-apply-combine pattern](http://pandas.pydata.org/pandas-docs/stable/groupby.html)
- Basic plotting


In [2]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

#### 1. Load the `chipotle.tsv` and examine the data.

In [3]:
chip_file = './datasets/chipotle.tsv'

chip_df = pd.read_csv(chip_file, delimiter='\t' )

The chipotle data is a .tsv, which means "tab separated value". This is just like a csv but the cells are separated by tabs. There is an argument in read_csv called delimiter, where you can specify the string that separates the tabs:

In [4]:
# A:

chip_df.head(10)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


#### 2. Create a sub-id for each order-id

We have an identifier for each order already in `order_id`, but no unique identifier for each _sub-order_ within the overall order.

Use grouping and `.apply()` to assign sub-ids for orders.

In [5]:
# A:


def add_sub_group(df):
    df['sub_group_id'] = np.arange(df.shape[0])
    return df

chip_df = chip_df.groupby('order_id').apply(add_sub_group)

chip_df.head()

,order_id,quantity,item_name,choice_description,item_price,sub_group_id
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39,0
1,1,1,Izze,[Clementine],$3.39,1
2,1,1,Nantucket Nectar,[Apple],$3.39,2
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39,3
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98,0


#### 3. Clean up the price column 

We want the price column to be a numeric float value. Currently it is a string (and has the dollar sign in it).

NameError: global name 'replace' is not defined

In [6]:
chip_df['item_price'] = chip_df.item_price.apply(lambda x: x.strip('$'))

    


In [7]:
chip_df['item_price'] = chip_df['item_price'].map(float)

chip_df.head()

,order_id,quantity,item_name,choice_description,item_price,sub_group_id
0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39,0
1,1,1,Izze,[Clementine],3.39,1
2,1,1,Nantucket Nectar,[Apple],3.39,2
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39,3
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,0


In [99]:
chip_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 6 columns):
order_id              4622 non-null int64
quantity              4622 non-null int64
item_name             4622 non-null object
choice_description    3376 non-null object
item_price            4622 non-null float64
sub_group_id          4622 non-null int32
dtypes: float64(1), int32(1), int64(2), object(2)
memory usage: 198.7+ KB


#### 4. Make a new categorical column for broader item type

Currently we have many different item types. Make a new column that only has 5 different broad item types. You should have these types in the new column in your DataFrame:

    chips
    drink
    burrito
    taco
    salad
    
(Put the `bowl` items into `burrito` category).

In [8]:
# A: create types column


#chip_df.groupby('item_name').size() - see names of items

def item_types(x):
    x = x.lower()
    if 'chips' in x:
        return 'chips'
    elif ('burrito' in x) or ('bowl' in x):
        return 'burrito'
    elif 'tacos' in x:
        return 'tacos'
    elif 'salad' in x:
        return 'salad'
    else:
        return 'drink'
    
chip_df['item_category'] = chip_df.item_name.map(item_types)    

chip_df.head()

,order_id,quantity,item_name,choice_description,item_price,sub_group_id,item_category
0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39,0,chips
1,1,1,Izze,[Clementine],3.39,1,drink
2,1,1,Nantucket Nectar,[Apple],3.39,2,drink
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39,3,chips
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,0,burrito


#### 5. Calculate the total price by `order_id` and add as a new as column `order_total_price`.

There are a variety of different ways you can tackle this problem. One way is a grouped apply on the price and then a merge by `order_id` with the total price.

Hints:

- Merging DataFrames with series doesn't work, you need to merge two DataFrames.
- A series object coming out of a groupby with an apply will have the groupby as potentially hierarchical indices. Using `reset_index()` will turn these back into columns and also convert to a DataFrame, which can be used to merge on.

In [9]:
# A: 

total_order_price = chip_df.groupby('order_id')['item_price'].apply(np.sum).reset_index()

total_order_price.rename(columns = {'item_price':'order_total_price'}, inplace = True)
total_order_price.head()

,order_id,order_total_price
0,1,11.56
1,2,16.98
2,3,12.67
3,4,21.00
4,5,13.70


In [102]:
chip_df = chip_df.merge(total_order_price, on='order_id')
print chip_df.shape
chip_df.head(10)

(4622, 8)


,order_id,quantity,item_name,choice_description,item_price,sub_group_id,item_category,order_total_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39,0,chips,11.56
1,1,1,Izze,[Clementine],3.39,1,drink,11.56
2,1,1,Nantucket Nectar,[Apple],3.39,2,drink,11.56
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39,3,chips,11.56
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,0,burrito,16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",10.98,0,burrito,12.67
6,3,1,Side of Chips,NaN,1.69,1,chips,12.67
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",11.75,0,burrito,21.00
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",9.25,1,tacos,21.00
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",9.25,0,burrito,13.70


#### 6. Make an `adjusted_item_price` column to account for multiple orders per row.

Some items have multiple orders per row, as indicated by the quantity. Adjust the price to account for the number of orders in a new column.

In [10]:
# A: price per item
chip_df['adjusted_item_price'] = chip_df.item_price / chip_df.quantity 

chip_df.head()

,order_id,quantity,item_name,choice_description,item_price,sub_group_id,item_category,adjusted_item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39,0,chips,2.39
1,1,1,Izze,[Clementine],3.39,1,drink,3.39
2,1,1,Nantucket Nectar,[Apple],3.39,2,drink,3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39,3,chips,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,0,burrito,8.49


#### 7. What is the min, max, count, mean and standard deviation of price for each unique item in  `item_name`?

Pivot table works well for this. You can pass multiple aggregation functions into the `aggfunc` argument.

The count won't just be the length of each order's subset - there are multiple orders per row sometimes (evidenced in the quantity column).

In [30]:
item_summary = pd.pivot_table(chip_df, index = 'item_name', values = ['adjusted_item_price'], 
               aggfunc=[np.min, np.max, np.mean, np.std, np.count_nonzero])

#is there a better way to add count 
item_summary.head()

,amin,amax,mean,std,count_nonzero
,adjusted_item_price,adjusted_item_price,adjusted_item_price,adjusted_item_price,adjusted_item_price
item_name,,,,,
6 Pack Soft Drink,6.49,6.49,6.490000,0.000000,54.0
Barbacoa Bowl,8.69,11.75,10.187273,1.260994,66.0
Barbacoa Burrito,8.69,11.75,9.832418,1.139519,91.0
Barbacoa Crispy Tacos,8.99,11.75,10.087273,1.270701,11.0
Barbacoa Salad Bowl,9.39,11.89,10.640000,1.317616,10.0


#### 8. Plot the mean price of items against the count (popularity).

You have this info in your summary table from the previous question.

In [10]:
# A:



#### 9. Plot the max price of items against the count (popularity).

In [11]:
# A:

### 10. Calculate the mean of adjusted price per broad category.

You can handle these with a single function if you want, or another way if you prefer.

Just FYI, apply functions can have keyword arguments that you pass in when you call the apply chained to the groupby.

for example:

```python
def my_applier(df, my_kwarg='placeholder'):
    df['newcol_'+placeholder] = 1.
    return df
    
data = data.groupby('variable').apply(my_applier, my_kwarg='colsuffix').reset_index(drop=True)
```


In [12]:
# A:

#### 11. Make a barplot of your price mean by the broad type category.


In [13]:
# A:

#### 12. [Challenge] Parse the `choice_description` column into two new columns: `order_customizations` and `order_customization_id`

Here is what your inputs and outputs would look like for a hypotheical section of the DataFrame (I'm just showing some of the columns to give you an idea of what the output format will be):

**Input:**

```python
                                  choice_description     item_name  order_id  \
0                                       [Clementine]          Izze         1   
1  [Red Salsa, [Black Beans, Guacamole, Sour Cream]]  Chicken Bowl         1   

   sub_order_id  
0             1  
1             2
```

**Output:**

```python
   order_customization_id order_customizations  \
0                       0           Clementine   
1                       0            Red Salsa   
2                       1          Black Beans   
3                       2            Guacamole   
4                       3           Sour Cream   

                                  choice_description     item_name  order_id  \
0                                       [Clementine]          Izze         1   
1  [Red Salsa, [Black Beans, Guacamole, Sour Cream]]  Chicken Bowl         1   
2  [Red Salsa, [Black Beans, Guacamole, Sour Cream]]  Chicken Bowl         1   
3  [Red Salsa, [Black Beans, Guacamole, Sour Cream]]  Chicken Bowl         1   
4  [Red Salsa, [Black Beans, Guacamole, Sour Cream]]  Chicken Bowl         1   

   sub_order_id  
0             1  
1             2  
2             2  
3             2  
4             2 
```

Hints:

- Remember you can write your own function and pass it into apply. In this case there will be one item per group, since we have to do this parsing for every row, but you may be able to get a solution with `.iterrows()` if you want to try that out.
- Within a function that you are passing into `.apply()`, you can create a _new DataFrame and return that._ This is one of the things that makes apply so powerful, since you can essentially perform any operations you want on a subset of your original DataFrame as long as you return DataFrames/groups that can be recombined.

- *Your output dataframe will be very long as there will be a row for every item in the item_name column.  Expect there to be alot of repeating information betweens rows, but they should not be exactly identical.*

**Note: the function may take awhile to complete. `apply` isn't that efficient with complicated custom operations per row like this.**

In [14]:
# A: